# Parking Analysis Calculations

This notebook retrieves parking data, processes it, and creates output files describing the findings.

In [22]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
from functools import reduce
from measurement.measures import Area
from decimal import Decimal
import folium
from shapely import geometry, ops
from string import Template
import datetime
import numpy as np
import os
%matplotlib inline
ox.config(log_console=True, use_cache=False)
ox.__version__

'0.11'

Specify the area to look for and the type of footprints to look for. Here, we only want the Downtown Neighborhood, and we are only looking for footprints tagged as "amenity:parking" in OSM.

**Note:** You must supply a `place` string which returns a valid result in OSM. For this calculation, I manually had to create the "Dowtown" neighborhood from the [City's GIS](http://pvdgis.maps.arcgis.com/home/webmap/viewer.html?useExisting=1&layers=07555d51a34a4fa0a43e9c920f68970f). Other neighborhoods may or may not be on OSM. For example, Fox Point and College Hill are, but Upper South Providence is not.

In [23]:
place = "Downtown, Providence, RI, USA"

We'll also download the Polygon representing the place's bounds, and get it's area (all areas are initially in square meters).

In [24]:
downtown_poly = ox.gdf_from_place(place).geometry[0]
downtown_area = ox.project_geometry(downtown_poly)[0].area

Now, we'll set up the `stats` `DataFrame` to hold a bunch of calculations:

In [25]:
stat_columns = {
    'count': np.int64,
    'disabled_count': np.int64,
    'area_sum': np.float64,
    'area_mean': np.float64,
    'area_median': np.float64,                  
    'area_percent_sum': np.float64,
    'area_percent_mean': np.float64,
    'area_percent_median': np.float64,
    'capacity_sum': np.int64,
    'capacity_mean': np.int64,
    'capacity_median': np.int64,
    'capacity:disabled_sum': np.int64,
    'capacity:disabled_mean': np.int64,
    'capacity:disabled_median': np.int64,
    'efficiency_sum': np.float64, 
    'efficiency_mean': np.float64,
    'efficiency_median': np.float64
}

stats = pd.DataFrame()
for c in stat_columns:
    stats[c] = pd.Series(dtype=stat_columns[c])

Because of the way OSM stores data and OSMnx retrieves it, we need to handle parking lots and garages separately from how we handle on-street parking.

## Lot/Garage Parking

We will use OSMnx to download "footprints." These are effectively polygons representing the shapes of lots in the neighborhood. Each shape is "tagged" (in OSM parlance) with various bits of information (surface type, capacity, etc.).

For OSMnx to retrieve the right footprints, we need to specify a type:

In [26]:
footprint_type = "parking"

Download the footprints from OSM, and them project them to UTM 19 via OSMnx (UTM Zone selected automatically):

In [27]:
footprints_unprojected = ox.footprints.footprints_from_place(place, footprint_type=footprint_type)
footprints = ox.project_gdf(footprints_unprojected)

We filter out uneeded information, filter out facilities without capacties set, calculate each footprint's area, and convert some values to numbers:

In [28]:
footprints = footprints[['name', 'parking', 'geometry', 'capacity', 'capacity:disabled']]
footprints = footprints.dropna(subset=['capacity'])
footprints['area'] = footprints.geometry.area
footprints['capacity'] = pd.to_numeric(footprints['capacity'])
footprints['capacity:disabled'] = pd.to_numeric(footprints['capacity:disabled'])

From this, we calculate the efficiency, the amount of space required per each parking spot, and a percentage of the region's land area each facility uses.

In [29]:
footprints['efficiency'] = footprints['area'] / footprints['capacity']
footprints['area_percent'] = 100 * (footprints['area'] / downtown_area)

Now, we filter the footprints into the subcategories. "Surface" lots, "structured" garages (indicated by the OSM `parking = multi-storey` tag, and "underground" garages.

In [30]:
fp_surface = footprints[footprints['parking'] == "surface"]
fp_structure = footprints[footprints['parking'] == "multi-storey"]
fp_underground = footprints[footprints['parking'] == "underground"]

Surface lots and strucutred lots have the same calculations:

In [31]:
for parking_type, frame in zip(['surface', 'structured'], [fp_surface, fp_structure]):
    stats.loc[parking_type, 'count'] = len(frame)
    stats.loc[parking_type, 'count_disabled'] = len(frame[frame['capacity:disabled'].notnull()])
    stats.loc[parking_type, 'area_sum'] = frame['area'].sum()
    stats.loc[parking_type, 'area_mean'] = frame['area'].mean()
    stats.loc[parking_type, 'area_median'] = frame['area'].median()
    stats.loc[parking_type, 'area_percent_sum'] = frame['area_percent'].sum()
    stats.loc[parking_type, 'area_percent_mean'] = frame['area_percent'].mean()
    stats.loc[parking_type, 'area_percent_median'] = frame['area_percent'].median()
    stats.loc[parking_type, 'capacity_sum'] = pd.to_numeric(frame['capacity']).sum()
    stats.loc[parking_type, 'capacity_mean'] = pd.to_numeric(frame['capacity']).mean()
    stats.loc[parking_type, 'capacity_median'] = pd.to_numeric(frame['capacity']).median()
    stats.loc[parking_type, 'capacity:disabled_sum'] = pd.to_numeric(frame['capacity:disabled']).sum()
    stats.loc[parking_type, 'capacity:disabled_mean'] = pd.to_numeric(frame['capacity:disabled']).mean()
    stats.loc[parking_type, 'capacity:disabled_median'] = pd.to_numeric(frame['capacity:disabled']).median()
    stats.loc[parking_type, 'efficiency_sum'] = frame['efficiency'].sum()
    stats.loc[parking_type, 'efficiency_mean'] = frame['efficiency'].mean()
    stats.loc[parking_type, 'efficiency_median'] = frame['efficiency'].median()

We don't calculate area for underground garages, so the calculations for this type are different.

In [32]:
parking_type = "underground"
frame = fp_underground

parking_dim_a_ft = 9
parking_dim_b_ft = 17
parking_spot_area = Area(sq_ft = parking_dim_a_ft * parking_dim_b_ft).sq_m

stats.loc[parking_type, 'count'] = len(frame)
stats.loc[parking_type, 'count_disabled'] = len(frame[frame['capacity:disabled'].notnull()])
stats.loc[parking_type, 'capacity_sum'] = pd.to_numeric(frame['capacity']).sum()
stats.loc[parking_type, 'capacity_mean'] = pd.to_numeric(frame['capacity']).mean()
stats.loc[parking_type, 'capacity_median'] = pd.to_numeric(frame['capacity']).median()
stats.loc[parking_type, 'capacity:disabled_sum'] = pd.to_numeric(frame['capacity:disabled']).sum()
stats.loc[parking_type, 'capacity:disabled_mean'] = pd.to_numeric(frame['capacity:disabled']).mean()
stats.loc[parking_type, 'capacity:disabled_median'] = pd.to_numeric(frame['capacity:disabled']).median()

## On-Street Parking

Now, we can focus on on-street parking. Streets are defined as "ways" in OSM. Ideally, we would be able to pull these from OSM using the `osmnx.pois` module, but this is not currently possible because `osmnx.pois.create_poi_gdf` filters the dataset using the OSM `amenity` tag, which won't work since street parking is defined with the `parking:lane` tag. **Note:** [there is discussion](https://github.com/gboeing/osmnx/pull/342) on changing this in a future release of OSMnx.

Instead, we will get *all* OSM data as JSON, and filter to the ways we want.

In [33]:
download = ox.osm_net_download(polygon=downtown_poly)[0]['elements']

def is_way_with_parking(element):
    match_tags = ['parking:lane:both', 'parking:lane:left', 'parking:lane:right']
    
    val = element['type'] == 'way'
    val &= 'tags' in element and any(k in match_tags for k in element['tags'].keys())
    
    return val

filtered = list(filter(is_way_with_parking, download))

street_parking_meta = pd.DataFrame()

for f in filtered:
    osmid = f['id']
    tags = f.get('tags', {})
    capacity = 0
    name = ""
    for t in tags.keys():
        if t.startswith('parking') and t.endswith('capacity'):
            capacity += int(tags[t])
        elif t == "name":
            name = tags[t]
    street_parking_meta.loc[osmid, 'name'] = name
    street_parking_meta.loc[osmid, 'capacity'] = int(capacity)

`street_parking_meta` is a plain `pandas.DataFrame`, with no geospatial information. We can now use `osmnx.graph_from_place` to pull the geospatial data for each of the streets we want.

By filtering the network by `parking:lane` criteria and merging the results with capacities stored in `street_parking_meta`, we generate the `street_parking` `GeoDataFrame` containing the geometries, street names, and capacities of different road segments.

In [35]:
def get_capacities(ids):
    if type(ids) == int:
        ids = [ids]
    total = 0
    for id in ids:
        total += street_parking_meta.loc[id, 'capacity']
    return int(total)

street_parking = gpd.GeoDataFrame()
street_parking_all_geometries = gpd.GeoDataFrame(columns=['geometry'])
street_parking_all_geometries.set_geometry('geometry')
c = 0

for filter_criteria in ["['parking:lane:left']", "['parking:lane:right']", "['parking:lane:both']"]:
    G = ox.graph_from_place(place, custom_filter=filter_criteria, retain_all=True, simplify=False)
    graph_gdf = ox.graph_to_gdfs(G, nodes=False, edges=True, node_geometry=False, fill_edge_geometry=True)
    graph_gdf['capacity'] = list(map(lambda x: get_capacities(x), graph_gdf['osmid']))
    if len(street_parking) == 0:
        street_parking = graph_gdf[graph_gdf['name'] == -1]
    for _, row in graph_gdf.iterrows():
        for k, v in row.iteritems():
            street_parking.loc[row['osmid'], k] = v
            street_parking_all_geometries.loc[c, k] = v
        c += 1
        
street_parking = street_parking[['capacity', 'name', 'geometry']]
street_parking_all_geometries.crs = street_parking.crs

/Users/chris/opt/anaconda3/envs/parking-analysis/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


This information can now be added to the `stats` `DataFrame`. For on-street parking, we assume average dimensions of 9 feet by 17 feet for a parking space to calculate area use.

In [36]:
parking_type = "street"
frame = street_parking
on_street_dim_a = 9
on_street_dim_b = 17
parking_spot_area = on_street_dim_a * on_street_dim_b

stats.loc[parking_type, 'capacity_sum'] = pd.to_numeric(frame['capacity']).sum()
stats.loc[parking_type, 'area_sum'] = Area(sq_ft=(stats.loc[parking_type, 'capacity_sum'] * parking_spot_area)).sq_m
stats.loc[parking_type, 'area_percent_sum'] = 100 * stats.loc[parking_type, 'area_sum'] / downtown_area
stats.loc[parking_type, 'efficiency_sum'] = stats.loc[parking_type, 'area_sum'] / stats.loc[parking_type, 'capacity_sum']

## Generating Output

These calculations are used to create an HTML document. A key component of that page is a [Leaflet](https://leafletjs.com) map showing parking locations and capacities. We generate that here with [Folium](https://python-visualization.github.io/folium/).

Start by creating the map and adding all the footprints:

In [37]:
footprints

,name,parking,geometry,capacity,capacity:disabled,area,efficiency,area_percent
121498413,Rhode Island Convention Center,multi-storey,"POLYGON ((299344.171 4633139.476, 299344.231 4...",2400,33.0,25484.379592,10.618491,1.262744
141535719,Biltmore Garage,multi-storey,"POLYGON ((299480.300 4633065.483, 299518.656 4...",440,NaN,3100.412970,7.046393,0.153625
141535724,Freeway Garage,multi-storey,"POLYGON ((299920.958 4632965.162, 299927.951 4...",300,NaN,1910.974236,6.369914,0.094688
141535769,Civic Center Garage,multi-storey,"POLYGON ((299345.605 4632950.568, 299388.869 4...",450,NaN,3264.253052,7.253896,0.161743
141535777,The Arcade Garage,multi-storey,"POLYGON ((299862.573 4633016.029, 299886.587 4...",600,NaN,1948.760916,3.247935,0.096561
...,...,...,...,...,...,...,...,...
767694425,NaN,surface,"POLYGON ((299370.095 4633167.528, 299371.381 4...",2,NaN,61.807201,30.903600,0.003063
767912677,Textron Building Garage,underground,"POLYGON ((299945.973 4633128.279, 299956.766 4...",85,NaN,1442.148282,16.966450,0.071458
767912678,Station Row Parking Garage,underground,"POLYGON ((299688.546 4633725.864, 299709.494 4...",169,NaN,5089.784499,30.117068,0.252197
10548330,NaN,surface,"POLYGON ((299627.073 4634244.053, 299634.871 4...",104,5.0,5890.200977,56.636548,0.291858


In [38]:
m = folium.Map(tiles='cartodbpositron')

a, b, c, d = downtown_poly.bounds
bbox = [[b, a], [d, c]]

m.fit_bounds(bbox)

footprints['area square ft'] = list(map(lambda x: Area(sq_m=x).sq_ft, footprints['area']))

surface_tooltip_fields = {
    'name': 'Lot Name',
    'parking': 'Parking Type',
    'area square ft': 'Area (ft<sup>2</sup>)',
    'capacity': 'Capacity',
    'capacity:disabled': 'Accessible Spaces'
}

multistorey_tooltip_fields = {
    'name': 'Garage Name',
    'parking': 'Parking Type',
    'area square ft': 'Area (ft<sup>2</sup>)',
    'capacity': 'Capacity',
    'capacity:disabled': 'Accessible Spaces'
}

underground_tooltip_fields = {
    'name': 'Garage Name',
    'parking': 'Parking Type',
    'capacity': 'Capacity',
    'capacity:disabled': 'Accessible Spaces'
}

colors = {
    'surface': 'darkred',
    'multi-storey': 'orangered',
    'underground': 'darkorange'
}

def style_function(x):
    color = colors.get(x['properties']['parking'], 'black')
    return {'fillColor': color, 'color': color, 'weight': 1, 'fillOpacity': 0.7}

def make_layer(parking_type, tooltip_fields, layer_name):
    color = colors.get(parking_type, 'black')
    tooltips = folium.features.GeoJsonTooltip(list(tooltip_fields.keys()), aliases=list(tooltip_fields.values()))
    footprints_layer = folium.GeoJson(footprints[footprints['parking'] == parking_type], style_function=style_function)
    footprints_layer.layer_name = "<span style='color:" + color + "'>" + layer_name + "</span>"
    footprints_layer.add_child(tooltips)
    footprints_layer.add_to(m)
    

layer_surface = ("surface", surface_tooltip_fields, "Surface Lots")
layer_structure = ("multi-storey", multistorey_tooltip_fields, "Structured Garages")
layer_underground = ("underground", underground_tooltip_fields, "Underground Garages")

layers_to_make = [layer_surface, layer_structure, layer_underground]
for l in layers_to_make:
    a, b, c = l
    make_layer(a, b, c)

/Users/chris/opt/anaconda3/envs/parking-analysis/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
/Users/chris/opt/anaconda3/envs/parking-analysis/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
/Users/chris/opt/anaconda3/envs/parking-analysis/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


Now, we'll add the streets:

In [39]:
street_tooltip_fields = {
    'name': 'Street Name',
    'capacity': 'Segment Parking Capacity'
}

def style_function(x):
    return {'color': 'CHOCOLATE', 'weight': 3, 'opacity': 0.7}

streets_layer = folium.GeoJson(street_parking_all_geometries, style_function=style_function)
tooltips = folium.features.GeoJsonTooltip(list(street_tooltip_fields.keys()), aliases=list(street_tooltip_fields.values()))
tooltips.add_to(streets_layer)
streets_layer.layer_name = "<span style='color:CHOCOLATE'>Street Parking</span>"
streets_layer.add_to(m)

/Users/chris/opt/anaconda3/envs/parking-analysis/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


Finally, we add a layer control, and the map is complete:

In [40]:
folium.LayerControl(collapsed=False, hideSingleBase=True).add_to(m)
m

We can save the map to an HTML document. Because we're hosting this project on GitHub and using GitHub Pages, we're putting all HTML into the `docs` directory.

In [41]:
m.save('../docs/map.html')
stats

,count,disabled_count,area_sum,area_mean,area_median,area_percent_sum,area_percent_mean,area_percent_median,capacity_sum,capacity_mean,capacity_median,capacity:disabled_sum,capacity:disabled_mean,capacity:disabled_median,efficiency_sum,efficiency_mean,efficiency_median,count_disabled
surface,193.0,NaN,253332.624902,1312.604274,747.536335,12.552561,0.065039,0.037040,8320.0,43.108808,27.0,164.0,2.928571,2.0,6102.002518,31.616593,28.947692,56.0
structured,16.0,NaN,86721.750504,5420.109406,3055.244677,4.297039,0.268565,0.151387,13451.0,840.687500,445.0,127.0,63.500000,63.5,139.746934,8.734183,6.804412,2.0
underground,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1832.0,166.545455,160.0,9.0,9.000000,9.0,NaN,NaN,NaN,1.0
street,NaN,NaN,30745.239155,NaN,NaN,1.523418,NaN,NaN,2163.0,NaN,NaN,NaN,NaN,NaN,14.214165,NaN,NaN,NaN


We'll use an HTML template located at `../docs/template.html` to build the main page with Python's `string.Template` class. This allows us to provide the values we've just calculated.

In [42]:
with open('../docs/template_detail.html', 'r') as t_file:
    t_html = Template(t_file.read())

last_updated = datetime.datetime.now().strftime("%d %B %Y")

dim_area_ft = on_street_dim_a * on_street_dim_b

extra_area_surface_ft = Area(sq_m=stats.loc['surface', 'efficiency_mean']).sq_ft - dim_area_ft
extra_area_surface_percent = extra_area_surface_ft / Area(sq_m=stats.loc['surface', 'efficiency_mean']).sq_ft

a = 0

output = t_html.substitute(
    date_updated=last_updated,
    downtown_area=str(round(Area(sq_m=downtown_area).sq_mi, 2)) + " miles<sup>2</sup>",
    
    total_street_spaces="{:,g}".format(stats.loc['street', 'capacity_sum']),
    
    on_street_dim_1=str(on_street_dim_a) + " feet",
    on_street_dim_2=str(on_street_dim_b) + " feet",
    
    on_street_area="{:,}".format(round(Area(sq_m=stats.loc['street', 'area_sum']).sq_ft, 2)) + " ft<sup>2</sup>",
    
    on_street_area_percent=str(round(stats.loc['street', 'area_percent_sum'], 2)) + "%",
    
    total_surface_spaces="{:,g}".format(stats.loc['surface', 'capacity_sum']),
    
    
    count_surface_lots="{:,g}".format(stats.loc['surface', 'count']),
    mean_surface_capacity=str(round(stats.loc['surface', 'capacity_mean'], 1)),
    median_surface_capacity="{:,g}".format(round(stats.loc['surface', 'capacity_median'])),
    
    total_surface_area_ft="{:,}".format(round(Area(sq_m=stats.loc['surface', 'area_sum']).sq_ft, 2)) + " ft<sup>2</sup>",
    total_surface_area_mi="{:,}".format(round(Area(sq_m=stats.loc['surface', 'area_sum']).sq_mi, 4)) + " mi<sup>2</sup>",
    
    surface_area_percent=str(round(stats.loc['surface', 'area_percent_sum'], 2)) + "%",
    
    mean_surface_area="{:,}".format(round(Area(sq_m=stats.loc['surface', 'area_mean']).sq_ft, 2)) + " ft<sup>2</sup>",
    median_surface_area="{:,}".format(round(Area(sq_m=stats.loc['surface', 'area_median']).sq_ft, 2)) + " ft<sup>2</sup>",
    
    area_per_surface_lot_space="{:,}".format(round(Area(sq_m=stats.loc['surface', 'efficiency_mean']).sq_ft), 2) + " ft<sup>2</sup>",
    
    dim_area=str(dim_area_ft) + " ft<sup>2</sup>",
    
    extra_area=str(round(extra_area_surface_ft, 2)) + " ft<sup>2</sup>",
    extra_percent = str(round(extra_area_surface_percent * 100, 2)) + "%",
    
    total_surface_disabled_spots="{:,g}".format(stats.loc['surface', 'capacity:disabled_sum']),
    count_lots_with_disabled="{:,g}".format(stats.loc['surface', 'count_disabled']),
    
    surface_disabled_percent_total=str(round(100 * stats.loc['surface', 'capacity:disabled_sum'] / stats.loc['surface', 'capacity_sum'], 2)) + "%",

    mean_dis_spaces=str(round(stats.loc['surface', 'capacity:disabled_mean'], 2)),
    median_dis_spaces=str(round(stats.loc['surface', 'capacity:disabled_median'], 0)),
   
    total_structured_spaces="{:,g}".format(stats.loc['structured', 'capacity_sum']),
    count_structured_lots=int(stats.loc['structured', 'count']),
    mean_structured_capacity="{:,g}".format(stats.loc['structured', 'capacity_mean']),
    median_structured_capacity="{:,g}".format(stats.loc['structured', 'capacity_median']),
    total_structured_area="{:,}".format(round(Area(sq_m=stats.loc['structured', 'area_sum']).sq_ft, 2)) + " ft<sup>2</sup>",
    mean_structured_area="{:,}".format(round(Area(sq_m=stats.loc['structured', 'area_mean']).sq_ft, 2)) + " ft<sup>2</sup>",
    median_structured_area="{:,}".format(round(Area(sq_m=stats.loc['structured', 'area_median']).sq_ft, 2)) + " ft<sup>2</sup>",
    strucutred_average_space_per_spot="{:,}".format(round((Area(sq_m=stats.loc['structured', 'efficiency_mean']).sq_ft), 2)) + " ft<sup>2</sup>",
    strucutred_efficiency_comparison=str(round(100 * ((stats.loc['surface', 'efficiency_mean'] - stats.loc['structured', 'efficiency_mean']) / stats.loc['surface', 'efficiency_mean']), 2)) + "%",
    
    total_structured_disabled_spots="{:,g}".format(stats.loc['structured', 'capacity:disabled_sum']),
    
    total_underground_capacity="{:,g}".format(stats.loc['underground', 'capacity_sum']),
    count_underground_lots="{:,g}".format(stats.loc['underground', 'count']),
    mean_underground_capacity="{:,g}".format(stats.loc['underground', 'capacity_mean']),
    median_underground_capacity="{:,g}".format(stats.loc['underground', 'capacity_median']),
    underground_capacity_disabled="{:,g}".format(stats.loc['underground', 'capacity:disabled_sum']),
    
    structured_area_percent=str(round(stats.loc['structured', 'area_percent_sum'], 2)) + "%",
    gt_lots="{:,g}".format(stats['count'].sum()),
    gt_capacity="{:,g}".format(stats['capacity_sum'].sum()),
    gt_dis_spaces="{:,g}".format(stats['capacity:disabled_sum'].sum()),
    gt_area="{:,}".format(round(Area(sq_m=stats['area_sum'].sum()).sq_mi, 4)) + " miles<sup>2</sup>",
    gt_perc="{:,}".format(round(stats['area_percent_sum'].sum(), 3)) + "%"
)
with open('../docs/detail.html', 'w+') as of:
    of.write(output)

with open('../docs/template_main.html', 'r') as t_file:
    t_html = Template(t_file.read())

output = t_html.substitute(
    total_surface_spaces="{:,g}".format(stats.loc['surface', 'capacity_sum']),
    total_street_spaces="{:,g}".format(stats.loc['street', 'capacity_sum']),
    total_structured_spaces="{:,g}".format(stats.loc['structured', 'capacity_sum']),
    total_underground_capacity="{:,g}".format(stats.loc['underground', 'capacity_sum']),
    date_updated=last_updated,
    gt_capacity="{:,g}".format(stats['capacity_sum'].sum()),
    gt_dis_spaces="{:,g}".format(stats['capacity:disabled_sum'].sum()),
    gt_area_ft="{:,}".format(int(Area(sq_m=stats['area_sum'].sum()).sq_ft)),
    gt_perc="{:,}".format(round(stats['area_percent_sum'].sum(), 3)) + "%"
)
with open('../docs/index.html', 'w+') as of:
    of.write(output)

We'll also export some frames referenced in the HTML as CSVs:

In [43]:
if not os.path.exists('../docs/files'):
    os.mkdir('../docs/files')

for fn, frame in zip(['stats.csv', 'fp_surface.csv', 'fp_structured.csv', 'fp_underground.csv', 'street_capacities.csv'],
                    [stats, fp_surface, fp_structure, fp_underground, street_parking_all_geometries]):
    with open('../docs/files/' + fn, 'w+') as of:
        of.write(frame.to_csv())